In [ ]:
!git clone https://github.com/marco-c/missing_symbols.git

In [1]:
import os
from datetime import datetime, timedelta
from pyspark.sql import functions

In [2]:
known_modules = set([module[:-4].lower() for module in os.listdir('missing_symbols/known_modules')])

In [3]:
num_days = 1
days = [datetime.utcnow().date() - timedelta(1) - timedelta(i) for i in range(0, num_days)]

In [4]:
dataset = SQLContext(sc).read.load(['s3://telemetry-parquet/socorro_crash/v1/crash_date=' + day.strftime('%Y%m%d') for day in days], 'parquet')

In [5]:
modules = dataset\
.filter(dataset['product'] == 'Firefox')\
.select(functions.explode(dataset['json_dump']['modules']).alias('module')).rdd\
.map(lambda v: v['module'])\
.filter(lambda m: m['missing_symbols'])\
.filter(lambda m: m['filename'].lower() not in known_modules)\
.flatMap(lambda m: [((m['filename'], m['version']), 1)])\
.reduceByKey(lambda x, y: x + y)\
.map(lambda v: (v[0][0], [(v[0][1], v[1])]))\
.reduceByKey(lambda x, y: x + y)\
.sortBy(lambda v: sum(count for ver,count in v[1]), ascending=False)\
.collect()

In [6]:
len(modules)

9927

In [7]:
modules[:30]

[(u'ntdll.dll',
  [(u'6.1.7601.17514', 7),
   (u'6.0.6000.16385', 2),
   (u'6.0.6002.24024', 7),
   (u'6.1.7601.23572', 2285),
   (u'6.3.14393.447', 91),
   (u'10.0.14965.1001', 123),
   (u'6.1.7048.0', 1),
   (u'5.2.3790.5830', 1),
   (u'6.1.7600.16384', 10),
   (u'6.1.7600.16385', 1),
   (u'6.1.7601.17124', 4),
   (u'6.2.14393.351', 5),
   (u'6.3.9600.17019', 2),
   (u'6.3.9600.16610', 20),
   (u'6.2.14955.1000', 1),
   (u'6.2.7989.0', 1),
   (u'10.0.14959.1000', 7),
   (u'10.0.14393.312', 1),
   (u'6.1.7601.18247', 1),
   (u'10.0.14393.0', 1),
   (u'10.0.14931.1000', 59),
   (u'6.1.7264.0', 2),
   (u'10.0.10586.672', 3723),
   (u'6.0.6002.24020', 1),
   (u'6.1.7077.0', 1),
   (u'6.1.7601.23543', 13490),
   (u'5.1.2600.3180', 5),
   (u'6.1.7601.16537', 4),
   (u'6.1.7057.0', 1),
   (u'5.1.2600.2180', 12),
   (u'6.2.10586.672', 27),
   (u'10.0.14942.1000', 1),
   (u'6.3.10586.672', 22),
   (u'10.0.14951.1000', 14),
   (u'10.0.14977.1000', 1),
   (u'6.3.10240.17184', 10),
   (u'6.1.720